<a href="https://colab.research.google.com/github/zahraDehghanian97/Poetry_Generator/blob/master/Char_Poem_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

In [5]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
embedding_dim = 256
rnn_units = 1024

# make data ready

In [6]:
filepath = "/content/drive/MyDrive/Colab Notebooks/shahname.txt"
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')
corpus = corpus.replace("\t"," \t ").replace("\n", " \n ")

In [7]:
vocab = sorted(set(corpus))
vocab_size = len(vocab)
print(len(vocab), "chars")
char2idx = {u:i for i, u in enumerate(vocab)}
print(char2idx)
idx2char = np.array(vocab)

48 chars
{'\t': 0, '\n': 1, ' ': 2, '(': 3, ')': 4, '«': 5, '»': 6, '،': 7, '؟': 8, 'ء': 9, 'آ': 10, 'أ': 11, 'ؤ': 12, 'ئ': 13, 'ا': 14, 'ب': 15, 'ت': 16, 'ث': 17, 'ج': 18, 'ح': 19, 'خ': 20, 'د': 21, 'ذ': 22, 'ر': 23, 'ز': 24, 'س': 25, 'ش': 26, 'ص': 27, 'ض': 28, 'ط': 29, 'ظ': 30, 'ع': 31, 'غ': 32, 'ف': 33, 'ق': 34, 'ل': 35, 'م': 36, 'ن': 37, 'ه': 38, 'و': 39, 'ٔ': 40, 'پ': 41, 'چ': 42, 'ژ': 43, 'ک': 44, 'گ': 45, 'ی': 46, '\u200c': 47}


In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

text_as_int = np.array([char2idx[c] for c in corpus])
seq_length = 1000
# examples_per_epoch = len(corpus)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
print(sequences)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (1001,), types: tf.int64>
<BatchDataset shapes: ((64, 1000), (64, 1000)), types: (tf.int64, tf.int64)>


# LSTM Model

In [9]:
def create_model_lstm(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE):
  model = keras.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model

In [10]:
lstm_model = create_model_lstm()
lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = lstm_model.fit(dataset, epochs=50)
main_lstm_model = create_model_lstm(batch_size=1)
main_lstm_model.set_weights(lstm_model.get_weights())
# main_lstm_model = tf.keras.models.load_model('char_lstm.h5')
main_lstm_model.summary()

Epoch 1/50
42/42 [==============================] - 178s 4s/step - loss: 12.0322 - accuracy: 0.2160
Epoch 2/50
42/42 [==============================] - 176s 4s/step - loss: 3.0005 - accuracy: 0.2733
Epoch 3/50
42/42 [==============================] - 174s 4s/step - loss: 2.4754 - accuracy: 0.3094
Epoch 4/50
42/42 [==============================] - 171s 4s/step - loss: 2.3729 - accuracy: 0.3366
Epoch 5/50
42/42 [==============================] - 172s 4s/step - loss: 2.2592 - accuracy: 0.3558
Epoch 6/50
42/42 [==============================] - 176s 4s/step - loss: 2.1934 - accuracy: 0.3711
Epoch 7/50
42/42 [==============================] - 178s 4s/step - loss: 2.1009 - accuracy: 0.3920
Epoch 8/50
42/42 [==============================] - 185s 4s/step - loss: 2.0265 - accuracy: 0.4077
Epoch 9/50
42/42 [==============================] - 193s 5s/step - loss: 1.9596 - accuracy: 0.4275
Epoch 10/50
42/42 [==============================] - 192s 5s/step - loss: 1.9014 - accuracy: 0.4434
Epoch 11

In [11]:
def generate_text(model, start_string, temperature = 0.6):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [12]:
print(generate_text(main_lstm_model, start_string=u"چنین گفت رستم به اسفندیار"))
main_lstm_model.save("char_lstm.h5")

چنین گفت رستم به اسفندیار 	 ز مرغی و کمند و هم چادر باغ 
 یکی جام سپاهی ز کین جاپیس 	 سربتر بندگان و شمشیر تو 
 جهاندار و کشتن ورا نیو بود 	 تنش را به کشت سزاوار نو 
 تو پاکیزکارت اکنون بسته شیر 	 کزیشی آر اندیشه‌هاز آز بخشت 
 ز تاج جهان با تو نبایدت گرد 	 بخواند مرا بنزدیک شخوب 
 جز از کهبان تخم کمت‌ور نام 	 همیشه سر سخت بدی برتاج 
 کنون من به یک دل پر از آهنین 	 نمانم هم اندر مهان گاوست 
 کمان را چنین و گوانی هراس 	 بگویم به تو هم ز پیوند و ستی 
 همی رای گردش را بگذرد دروغ 	 میان مادرش آگه اندر صطرخروا 
 به رزم اندرون گیتی اندر آورد 	 سپردش ز آنگها به کردار شید 
 هرانگه که دارند گردان آفرین 	 بران آرزوی را جهانجوی شیر جای داد 
 بپوشید رویین هم از راست گاه 	 همرش ستمست خروش از پاسبان 
 ز خانه مر او را شهد قارن روم 	 شب و روز پارس و ز تن جای آورد 
 کجا رفت بیدار بدخواه و گفت 	 که از من بارت پیمان یادگار 
 کنون زان روی هامون بداد اوست 	 که هومنش و درکش هم آورد شاه 
 که دانش بردوار ایران بخواند 	 پرستنده شدند آن زمان و بس اندپی تن 	 که بر تخت زرین و مهر ژیان 
 فرستاد خاقان چنین شد پاسخ 	